In [31]:
from bs4 import BeautifulSoup
import re
import pandas as pd

def preprocess_html_message(html_text):
    if pd.isna(html_text):
        return ""  # 또는 NaN 유지하려면: return np.nan

    # BeautifulSoup으로 HTML 파싱
    soup = BeautifulSoup(html_text, "html.parser")

    # <br> 태그는 줄바꿈으로 대체
    for br in soup.find_all("br"):
        br.replace_with("\n")

    # 텍스트만 추출
    raw_text = soup.get_text(separator=" ", strip=True)

    # 공백 정리
    raw_text = re.sub(r'\s+', ' ', raw_text)

    return raw_text


In [32]:
# 데이터 불러오기기

df = pd.read_excel(r"C:\Users\user\Desktop\NER\myfair_participation_message_1982.xlsx").dropna(how='all')

print(df.head())

        id                                            message process_state  \
0  43031.0  <div class="css-0"><br data-mce-bogus="1"></di...   부스 예약 접수 완료   
1  43034.0  <p>(고객사 담당자명 직함)님, 안녕하세요</p><p>마이페어 (마이페어 담당자명...      참가 품목 안내   
2  43435.0  <p>안녕하세요, 마이페어입니다.</p><p><br data-mce-bogus="1...    부스 포함사항 안내   
3  43516.0  <p>담당자님 안녕하세요 보내주신 내용 확인했습니다<br>부재중이셔서 채팅드립니다<...           NaN   
4  43532.0  <p>네 담당자님 확인했습니다!&nbsp;혹시 부스 자리 선택 또는 2면 오픈 부스...           NaN   

            sender_email message_type  participation_num  
0  customer@customer.com        ADMIN               1982  
1       myfair@myfair.co        ADMIN               1982  
2       myfair@myfair.co        ADMIN               1982  
3  customer@customer.com     CUSTOMER               1982  
4  customer@customer.com     CUSTOMER               1982  


In [33]:
# 전처리 함수 적용
df['message_clean'] = df['message'].apply(preprocess_html_message)

# 결과 출력력
print(df[['message', 'message_clean']].head())

                                             message  \
0  <div class="css-0"><br data-mce-bogus="1"></di...   
1  <p>(고객사 담당자명 직함)님, 안녕하세요</p><p>마이페어 (마이페어 담당자명...   
2  <p>안녕하세요, 마이페어입니다.</p><p><br data-mce-bogus="1...   
3  <p>담당자님 안녕하세요 보내주신 내용 확인했습니다<br>부재중이셔서 채팅드립니다<...   
4  <p>네 담당자님 확인했습니다!&nbsp;혹시 부스 자리 선택 또는 2면 오픈 부스...   

                                       message_clean  
0  🎉 부스예약이 접수되었습니다. 마이페어에서 참가신청을 시작해 주셔서 감사합니다. 서...  
1  (고객사 담당자명 직함)님, 안녕하세요 마이페어 (마이페어 담당자명)입니다. 비품이...  
2  안녕하세요, 마이페어입니다. 답변 기다려 주셔서 감사합니다. 부스패키지는 아래와 같...  
3  담당자님 안녕하세요 보내주신 내용 확인했습니다 부재중이셔서 채팅드립니다 부스 안에 ...  
4  네 담당자님 확인했습니다! 혹시 부스 자리 선택 또는 2면 오픈 부스 선택 가능한가...  


In [36]:
df['message_clean'][150]

'안녕하세요, (고객사 담당자명 직함)님. 마이페어 (마이페어 담당자명 호칭)입니다. 네! 지금 바로 하실 수 있게 도와드렸습니다! 해당 절차 이후, 별도로 해주실 건 없으시며 정산에 필요한 요청 드렸던 서류 전달 주시면 됩니다. 이후로는 저희가 정산기관에 모든 서류들 모아서 제출하고 정산 받겠습니다. (약 1~2개월 소요) 감사합니다.'

In [37]:
import torch
from transformers import MarianMTModel, MarianTokenizer

# 모델 로딩
model_name = "Helsinki-NLP/opus-mt-ko-en"
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)

# 번역 함수 정의
def translate_korean_to_english(text):
    if pd.isna(text) or text.strip() == "":
        return ""
    
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        translated = model.generate(**inputs)
    english_text = tokenizer.decode(translated[0], skip_special_tokens=True)
    return english_text


# DataFrame에 적용
df['message_en'] = df['message_clean'].apply(translate_korean_to_english)

# 결과 확인
print(df[['message_clean', 'message_en']].head())

c:\Users\user\anaconda3\envs\ner\lib\site-packages\transformers\models\marian\tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


                                       message_clean  \
0  🎉 부스예약이 접수되었습니다. 마이페어에서 참가신청을 시작해 주셔서 감사합니다. 서...   
1  (고객사 담당자명 직함)님, 안녕하세요 마이페어 (마이페어 담당자명)입니다. 비품이...   
2  안녕하세요, 마이페어입니다. 답변 기다려 주셔서 감사합니다. 부스패키지는 아래와 같...   
3  담당자님 안녕하세요 보내주신 내용 확인했습니다 부재중이셔서 채팅드립니다 부스 안에 ...   
4  네 담당자님 확인했습니다! 혹시 부스 자리 선택 또는 2면 오픈 부스 선택 가능한가...   

                                          message_en  
0  Thank you for starting to apply for this servi...  
1  Hi, it's Mapeer. I'll take you to the organize...  
2            Hi, it's Mafeer. Thank you for waiting.  
3  Hi, sir. I've checked the information you sent...  
4  I'll call you after I deposit, pay you, pay yo...  


In [39]:
df = df.dropna()
df

,id,message,process_state,sender_email,message_type,participation_num,message_clean,message_en
0,43031.0,"<div class=""css-0""><br data-mce-bogus=""1""></di...",부스 예약 접수 완료,customer@customer.com,ADMIN,1982,🎉 부스예약이 접수되었습니다. 마이페어에서 참가신청을 시작해 주셔서 감사합니다. 서...,Thank you for starting to apply for this servi...
1,43034.0,"<p>(고객사 담당자명 직함)님, 안녕하세요</p><p>마이페어 (마이페어 담당자명...",참가 품목 안내,myfair@myfair.co,ADMIN,1982,"(고객사 담당자명 직함)님, 안녕하세요 마이페어 (마이페어 담당자명)입니다. 비품이...","Hi, it's Mapeer. I'll take you to the organize..."
2,43435.0,"<p>안녕하세요, 마이페어입니다.</p><p><br data-mce-bogus=""1...",부스 포함사항 안내,myfair@myfair.co,ADMIN,1982,"안녕하세요, 마이페어입니다. 답변 기다려 주셔서 감사합니다. 부스패키지는 아래와 같...","Hi, it's Mafeer. Thank you for waiting."
7,43527.0,"<p>안녕하세요, 마이페어입니다.</p><p><br data-mce-bogus=""1...",부스 포함사항 안내 (1),myfair@myfair.co,ADMIN,1982,"안녕하세요, 마이페어입니다. 전화 오신 줄 몰랐네요...! 못 받은 점 양해 부탁드...","Hi, Myperer. I didn't know you were calling. I..."
8,43529.0,"<p>안녕하세요, 마이페어입니다.</p><p><br data-mce-bogus=""1...",수출바우처 구매 금액 안내,myfair@myfair.co,ADMIN,1982,"안녕하세요, 마이페어입니다. 마이페어서비스 바우처형 Basic 만 이용해주시는거기 ...","Hi, it's Mafeerre, and it's only due to the us..."
...,...,...,...,...,...,...,...,...
157,63094.0,"<p style="""">안녕하세요, (고객사 담당자명 직함)님.</p><p style...",참가 안내 (1),myfair@myfair.co,ADMIN,1982,"안녕하세요, (고객사 담당자명 직함)님. 마이페어 (마이페어 담당자명 호칭)입니다....","Hi, I'm Maypeer, and I've checked your papers,..."
159,63110.0,"<p style="""">안녕하세요, (고객사 담당자명 직함)님.</p><p style...",정산 안내,myfair@myfair.co,ADMIN,1982,"안녕하세요, (고객사 담당자명 직함)님. 마이페어 (마이페어 담당자명 호칭)입니다....","Hi, it's Mapeer, it's a Maypeer, it's a Maypee..."
161,63121.0,"<p style="""">안녕하세요, (고객사 담당자명 직함)님.</p><p style...",정산 안내 (1),myfair@myfair.co,ADMIN,1982,"안녕하세요, (고객사 담당자명 직함)님. 마이페어 (마이페어 담당자명 호칭)입니다....","Hello, Mr. Maypeer. Yes! Thank you."
162,66016.0,"<p style="""">안녕하세요, (고객사 담당자명 직함)님.</p><p style...",수출바우처 정산 완료 안내,myfair@myfair.co,ADMIN,1982,"안녕하세요, (고객사 담당자명 직함)님. 마이페어 (마이페어 담당자명 호칭)입니다....","Hi, I'm Mapeer. I've got a deposit on a gold d..."


In [41]:
df.to_excel(r"C:\Users\user\Desktop\NER\myfair_participation_message_2268_en.xlsx", index=False)

In [35]:
from flair.data import Sentence
from flair.models import SequenceTagger
from transformers import MarianMTModel, MarianTokenizer

tagger = SequenceTagger.load("ner-ontonotes")

2025-04-24 22:46:21,157 SequenceTagger predicts: Dictionary with 75 tags: O, S-PERSON, B-PERSON, E-PERSON, I-PERSON, S-GPE, B-GPE, E-GPE, I-GPE, S-ORG, B-ORG, E-ORG, I-ORG, S-DATE, B-DATE, E-DATE, I-DATE, S-CARDINAL, B-CARDINAL, E-CARDINAL, I-CARDINAL, S-NORP, B-NORP, E-NORP, I-NORP, S-MONEY, B-MONEY, E-MONEY, I-MONEY, S-PERCENT, B-PERCENT, E-PERCENT, I-PERCENT, S-ORDINAL, B-ORDINAL, E-ORDINAL, I-ORDINAL, S-LOC, B-LOC, E-LOC, I-LOC, S-TIME, B-TIME, E-TIME, I-TIME, S-WORK_OF_ART, B-WORK_OF_ART, E-WORK_OF_ART, I-WORK_OF_ART, S-FAC
